In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('patent_abstract_5000_2015.csv')
df

,Unnamed: 0,publication_number,application_number,text,filing_date,new_appl_nbr,appln_id,app_nbr,filing,tech_field,...,breakthrough,breakthrough_xy,generality,originality,radicalness,renewal,quality_index_4,quality_index_6,quality_rank,tech_field_big_cat
0,201997,EP-3085354-A1,EP-16167755-A,Provided herein are methods of using gaseous n...,20111215,EP20160167755,451813367,EP20160167755,2011,16.0,...,NaN,NaN,0.000000,0.880658,0.240741,8.0,0.177162,0.167842,0,2
1,27936,EP-2510698-A2,EP-10836203-A,Disclosed is a method and apparatus of encodin...,20101208,EP20100836203,334696626,EP20100836203,2010,2.0,...,NaN,NaN,0.000000,0.690086,0.156627,9.0,0.382812,0.432070,1,0
2,718991,EP-2927353-A2,EP-15162415-A,The invention relates to an air-jet spinning m...,20150402,EP20150162415,438662266,EP20150162415,2015,28.0,...,NaN,NaN,0.000000,NaN,0.000000,3.0,0.304487,0.246568,0,3
3,322964,EP-2731905-A1,EP-12814169-A,A dispensing closure having a closure body and...,20120716,EP20120814169,380483874,EP20120814169,2012,25.0,...,NaN,NaN,NaN,0.744898,0.785714,6.0,0.127407,0.235815,0,3
4,788590,EP-2979831-A1,EP-15178742-A,A slitting apparatus having an anvil cylinder ...,20150728,EP20150178742,443003052,EP20150178742,2015,26.0,...,NaN,NaN,NaN,0.857143,0.392857,3.0,0.338889,0.459559,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,214061,EP-2539441-A1,EP-11704979-A,The present invention relates to a cell cultur...,20110221,EP20110704979,332527837,EP20110704979,2011,15.0,...,NaN,NaN,NaN,0.765306,0.571429,8.0,0.275205,0.231471,0,2
4996,20029,EP-2483785-A2,EP-10822339-A,The present disclosure includes methods and de...,20100920,EP20100822339,333404856,EP20100822339,2010,6.0,...,NaN,NaN,0.408163,0.777778,0.208333,8.0,0.338130,0.322977,1,0
4997,636303,EP-2845982-A2,EP-14184100-A,The present invention relates to a post 1 for ...,20140909,EP20140184100,421699870,EP20140184100,2014,35.0,...,NaN,NaN,NaN,0.639053,0.230769,4.0,0.182051,0.240659,0,4
4998,85513,EP-2470961-A1,EP-10752443-A,Within an area (A 0 ) where of four heads (60 ...,20100824,EP20100752443,323501678,EP20100752443,2010,9.0,...,NaN,NaN,NaN,0.848837,0.639535,8.0,0.381746,0.422516,1,1


In [48]:
conditions = [

    (df['quality_index_4'] < 0.3),
    (df['quality_index_4'] >= 0.3)
]

choices = ['0', '1']

In [49]:

df['quality_rank'] = np.select(condlist=conditions, choicelist=choices)

In [50]:
df = df[['text','quality_rank']]

In [51]:
df['text'] = df['text'].apply(str)

/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Create a model

In [52]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

In [53]:
# more CNN library
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


In [54]:
MAX_SEQUENCE_LENGTH = 500  # [Steven] I added for the CNN, to take only the 500 words.
MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000  # [Steven] this should be number of unique word / vocabulary
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [55]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", str(string))    
    string = re.sub(r"\'", "", str(string))    
    string = re.sub(r"\"", "", str(string))    
    return string.strip().lower()

In [56]:
data_train = df.sample(frac=1, random_state = 5)[:5000]
data_train

,text,quality_rank
27,The invention relates to a device 20 for detec...,0
1482,A telescopic extension device for cleaning sys...,0
3021,The invention relates to a low-voltage switchi...,0
3867,The invention relates to a method for manufact...,1
637,Embodiments of the present invention disclose ...,0
...,...,...
3046,The present invention relates to a vehicle ped...,0
1725,In a direct drill bit drive for tools for comm...,1
4079,The present invention relates to a computer-im...,0
2254,The invention relates to a device (25) for pro...,0


In [57]:
data_train.shape

(5000, 2)

In [58]:
import nltk
from nltk import tokenize

abstracts = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.text.shape[0]): # for each row
    #text = BeautifulSoup(data_train.text[idx])
    #print(clean_str(str(data_train.iloc[idx]['text'])))
    text = clean_str(str(data_train.iloc[idx]['text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence of one abstract (but it's just an intermediate variable, not used directly in later code)
    abstracts.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data) (this is what we use )
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [59]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
# data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [60]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 17545 unique tokens.


In [61]:
len(sequences)

5000

In [62]:
#sequences

In [63]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post') # The default is pre-padding, should I try post-padding ? => padding='post', truncating='post'. I think it has to do with whether the beginning words are more important, or later words more important. You may want to try both approaches and compare result?
data

array([[   1,   26,   42, ...,    0,    0,    0],
       [   2, 6771, 1069, ...,    0,    0,    0],
       [   1,   26,   42, ...,    0,    0,    0],
       ...,
       [   1,   54,   26, ...,    0,    0,    0],
       [   1,   26,   42, ...,    0,    0,    0],
       [  85,    9, 6135, ...,    0,    0,    0]], dtype=int32)

In [64]:
data.shape

(5000, 500)

In [65]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 500)
Shape of label tensor: (5000, 2)


In [66]:
indices.shape

(5000,)

In [67]:
# I've already shuffle above using data_train = df.sample(frac=1, random_state = 5)[:5000]

# split the data into a training set and a validation set
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
num_validation_samples

1000

In [68]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))


Number of positive and negative reviews in traing and validation set
[2632. 1368.]
[651. 349.]


In [69]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [70]:
y_val

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [71]:
import os
os.getcwd()

'/home/lin_menghsien/team_repo'

In [72]:
# import GloVe word embedding
GLOVE_DIR = "" #"home/lin_menghsien/w266/project/data/glove" # I save the glove file in the current team_repo directory
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [73]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [74]:
len(word_index) + 1

17546

In [75]:
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))  # EMBEDDING_DIM = 100 since we import 100d GloVe
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [76]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [77]:
import os, sys, re, json, time, datetime, shutil
from numpy.random import seed
seed(5)
import tensorflow
tensorflow.random.set_seed(5)

In [78]:
# Parameter setting
epochs = 15
embed_dim = EMBEDDING_DIM  # already specify with EMBEDDING_DIM variable above
dense_layer_dims = [100]
dropout_rate = 0.2
num_filters = [25, 25, 50, 50, 50]
kernel_sizes = [3, 4, 15, 50, 150]
num_classes = labels.shape[1] # 

# Model building
start_time = time.time()
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
h = embedding_layer(sequence_input) # remember embedding_layer assign with GloVe's weight

conv_layers_for_all_kernel_sizes = []
counter = 0
for kernel_size, filters in zip(kernel_sizes, num_filters): 
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
    counter += 1
    #print(conv_layers_for_all_kernel_sizes)
if counter == 1: # need this since layers.concatenate throw error when there is only 1 item in conv_layers_for_all_kernel_sizes
    h = conv_layer
else:
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1) # Concat the feature maps from each different size.

    h = keras.layers.Dropout(rate=dropout_rate)(h) # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values in the vector

for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation = 'relu')(h) # [Steven] I believe this add additional fully-connected hidden layer with the hope to increase model accuracy

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=sequence_input, outputs=prediction)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

# train
model.reset_states()
model.fit(x_train, 
          y_train, 
          epochs=epochs, 
          validation_data=(x_val, y_val))


Epoch 1/15
125/125 [==============================] - 79s 629ms/step - loss: 0.7134 - accuracy: 0.6212 - val_loss: 0.6522 - val_accuracy: 0.6510
Epoch 2/15
125/125 [==============================] - 77s 619ms/step - loss: 0.6051 - accuracy: 0.6697 - val_loss: 0.6744 - val_accuracy: 0.5750
Epoch 3/15
125/125 [==============================] - 78s 628ms/step - loss: 0.5223 - accuracy: 0.7440 - val_loss: 0.7634 - val_accuracy: 0.6510
Epoch 4/15
125/125 [==============================] - 75s 601ms/step - loss: 0.4189 - accuracy: 0.8125 - val_loss: 0.6980 - val_accuracy: 0.6180
Epoch 5/15
125/125 [==============================] - 72s 579ms/step - loss: 0.2049 - accuracy: 0.9287 - val_loss: 1.0600 - val_accuracy: 0.6550
Epoch 6/15
125/125 [==============================] - 72s 573ms/step - loss: 0.1270 - accuracy: 0.9572 - val_loss: 1.0802 - val_accuracy: 0.4520
Epoch 7/15
125/125 [==============================] - 73s 581ms/step - loss: 0.0779 - accuracy: 0.9750 - val_loss: 1.1016 - val_ac

In [79]:
# Parameter setting
epochs = 15
embed_dim = EMBEDDING_DIM  # already specify with EMBEDDING_DIM variable above
dense_layer_dims = [100]
dropout_rate = 0.2
num_filters = [100, 100, 100]
kernel_sizes = [48, 50, 52]
num_classes = labels.shape[1] # 

# Model building
start_time = time.time()
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
h = embedding_layer(sequence_input) # remember embedding_layer assign with GloVe's weight

conv_layers_for_all_kernel_sizes = []
counter = 0
for kernel_size, filters in zip(kernel_sizes, num_filters): 
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
    counter += 1
    #print(conv_layers_for_all_kernel_sizes)
if counter == 1: # need this since layers.concatenate throw error when there is only 1 item in conv_layers_for_all_kernel_sizes
    h = conv_layer
else:
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1) # Concat the feature maps from each different size.

    h = keras.layers.Dropout(rate=dropout_rate)(h) # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values in the vector

for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation = 'relu')(h) # [Steven] I believe this add additional fully-connected hidden layer with the hope to increase model accuracy

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=sequence_input, outputs=prediction)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

# train
model.reset_states()
model.fit(x_train, 
          y_train, 
          epochs=epochs, 
          validation_data=(x_val, y_val))


Epoch 1/15
125/125 [==============================] - 84s 673ms/step - loss: 0.8087 - accuracy: 0.6087 - val_loss: 0.6514 - val_accuracy: 0.6510
Epoch 2/15
125/125 [==============================] - 83s 662ms/step - loss: 0.5996 - accuracy: 0.6712 - val_loss: 0.6621 - val_accuracy: 0.6200
Epoch 3/15
125/125 [==============================] - 82s 657ms/step - loss: 0.4876 - accuracy: 0.7740 - val_loss: 0.8527 - val_accuracy: 0.6510
Epoch 4/15
125/125 [==============================] - 81s 652ms/step - loss: 0.2907 - accuracy: 0.8913 - val_loss: 0.7298 - val_accuracy: 0.6170
Epoch 5/15
125/125 [==============================] - 80s 641ms/step - loss: 0.1328 - accuracy: 0.9582 - val_loss: 0.9314 - val_accuracy: 0.5190
Epoch 6/15
125/125 [==============================] - 79s 633ms/step - loss: 0.1328 - accuracy: 0.9485 - val_loss: 0.9492 - val_accuracy: 0.6040
Epoch 7/15
125/125 [==============================] - 78s 626ms/step - loss: 0.0575 - accuracy: 0.9830 - val_loss: 1.0666 - val_ac

In [80]:
# Parameter setting
epochs = 15
embed_dim = EMBEDDING_DIM  # already specify with EMBEDDING_DIM variable above
dense_layer_dims = [100]
dropout_rate = 0.2
num_filters = [50, 50, 50, 50]
kernel_sizes = [5, 20, 55, 125]
num_classes = labels.shape[1] # 

# Model building
start_time = time.time()
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
h = embedding_layer(sequence_input) # remember embedding_layer assign with GloVe's weight

conv_layers_for_all_kernel_sizes = []
counter = 0
for kernel_size, filters in zip(kernel_sizes, num_filters): 
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
    counter += 1
    #print(conv_layers_for_all_kernel_sizes)
if counter == 1: # need this since layers.concatenate throw error when there is only 1 item in conv_layers_for_all_kernel_sizes
    h = conv_layer
else:
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1) # Concat the feature maps from each different size.

    h = keras.layers.Dropout(rate=dropout_rate)(h) # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values in the vector

for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation = 'relu')(h) # [Steven] I believe this add additional fully-connected hidden layer with the hope to increase model accuracy

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=sequence_input, outputs=prediction)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

# train
model.reset_states()
model.fit(x_train, 
          y_train, 
          epochs=epochs, 
          validation_data=(x_val, y_val))


Epoch 1/15
125/125 [==============================] - 69s 551ms/step - loss: 0.7397 - accuracy: 0.6012 - val_loss: 0.6475 - val_accuracy: 0.6490
Epoch 2/15
125/125 [==============================] - 69s 550ms/step - loss: 0.6156 - accuracy: 0.6653 - val_loss: 0.6459 - val_accuracy: 0.6400
Epoch 3/15
125/125 [==============================] - 68s 541ms/step - loss: 0.5307 - accuracy: 0.7380 - val_loss: 0.6547 - val_accuracy: 0.6380
Epoch 4/15
125/125 [==============================] - 68s 545ms/step - loss: 0.4135 - accuracy: 0.8115 - val_loss: 0.7043 - val_accuracy: 0.5770
Epoch 5/15
125/125 [==============================] - 68s 544ms/step - loss: 0.1943 - accuracy: 0.9367 - val_loss: 0.8352 - val_accuracy: 0.5330
Epoch 6/15
125/125 [==============================] - 69s 551ms/step - loss: 0.1677 - accuracy: 0.9370 - val_loss: 1.8176 - val_accuracy: 0.3550
Epoch 7/15
125/125 [==============================] - 69s 548ms/step - loss: 0.1400 - accuracy: 0.9465 - val_loss: 0.9766 - val_ac

In [81]:
# Parameter setting
epochs = 15
embed_dim = EMBEDDING_DIM  # already specify with EMBEDDING_DIM variable above
dense_layer_dims = [100]
dropout_rate = 0.2
num_filters = [100, 100, 100]
kernel_sizes = [5, 50, 150]
num_classes = labels.shape[1] # 

# Model building
start_time = time.time()
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
h = embedding_layer(sequence_input) # remember embedding_layer assign with GloVe's weight

conv_layers_for_all_kernel_sizes = []
counter = 0
for kernel_size, filters in zip(kernel_sizes, num_filters): 
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
    counter += 1
    #print(conv_layers_for_all_kernel_sizes)
if counter == 1: # need this since layers.concatenate throw error when there is only 1 item in conv_layers_for_all_kernel_sizes
    h = conv_layer
else:
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1) # Concat the feature maps from each different size.

    h = keras.layers.Dropout(rate=dropout_rate)(h) # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values in the vector

for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation = 'relu')(h) # [Steven] I believe this add additional fully-connected hidden layer with the hope to increase model accuracy

prediction = keras.layers.Dense(num_classes, activation='softmax')(h)

model = keras.Model(inputs=sequence_input, outputs=prediction)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.

# train
model.reset_states()
model.fit(x_train, 
          y_train, 
          epochs=epochs, 
          validation_data=(x_val, y_val))


Epoch 1/15
125/125 [==============================] - 92s 737ms/step - loss: 0.7335 - accuracy: 0.6152 - val_loss: 0.6584 - val_accuracy: 0.6510
Epoch 2/15
125/125 [==============================] - 93s 740ms/step - loss: 0.6127 - accuracy: 0.6668 - val_loss: 0.6508 - val_accuracy: 0.6380
Epoch 3/15
125/125 [==============================] - 93s 742ms/step - loss: 0.5258 - accuracy: 0.7393 - val_loss: 0.9243 - val_accuracy: 0.6520
Epoch 4/15
125/125 [==============================] - 91s 728ms/step - loss: 0.3800 - accuracy: 0.8380 - val_loss: 0.7439 - val_accuracy: 0.5790
Epoch 5/15
125/125 [==============================] - 92s 737ms/step - loss: 0.1510 - accuracy: 0.9530 - val_loss: 0.8543 - val_accuracy: 0.5800
Epoch 6/15
125/125 [==============================] - 92s 736ms/step - loss: 0.1092 - accuracy: 0.9638 - val_loss: 1.0668 - val_accuracy: 0.6320
Epoch 7/15
125/125 [==============================] - 89s 714ms/step - loss: 0.0800 - accuracy: 0.9728 - val_loss: 1.1525 - val_ac